# Access data on Azure Data Lake Storage Gen2 (ADLS Gen2) with Synapse Spark

Azure Data Lake Storage Gen2 (ADLS Gen2) is used as the storage account associated with a Synapse workspace. A synapse workspace can have a default ADLS Gen2 storage account and additional linked storage accounts. 

You can access data on ADLS Gen2 with Synapse Spark via following URL:
    
    abfss://<container_name>@<storage_account_name>.dfs.core.windows.net/<path>

This notebook provides examples of how to read data from ADLS Gen2 account into a Spark context and how to write the output of Spark jobs directly into an ADLS Gen2 location.

## Pre-requisites
Synapse leverage AAD pass-through to access any ADLS Gen2 account (or folder) to which you have a **Blob Storage Contributor** permission. No credentials or access token is required. 

## Load a sample data

Let's first load the [public holidays](https://azure.microsoft.com/en-us/services/open-datasets/catalog/public-holidays/) of last 6 months from Azure Open datasets as a sample.

In [1]:
from azureml.opendatasets import PublicHolidays

from datetime import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta


end_date = datetime.today()
start_date = datetime.today() - relativedelta(months=6)
hol = PublicHolidays(start_date=start_date, end_date=end_date)
hol_df = hol.to_spark_dataframe()

StatementMeta(SampleSpark, 18, 1, Finished, Available)

In [3]:
# Display 5 rows
hol_df.show(5, truncate = False)

StatementMeta(SampleSpark, 18, 3, Finished, Available)

+---------------+----------------------------+----------------------------+-------------+-----------------+-------------------+
|countryOrRegion|holidayName                 |normalizeHolidayName        |isPaidTimeOff|countryRegionCode|date               |
+---------------+----------------------------+----------------------------+-------------+-----------------+-------------------+
|Norway         |Søndag                      |Søndag                      |null         |NO               |2020-11-29 00:00:00|
|Sweden         |Söndag                      |Söndag                      |null         |SE               |2020-11-29 00:00:00|
|Scotland       |St. Andrew's Day            |St. Andrew's Day            |null         |null             |2020-11-30 00:00:00|
|United Kingdom |St. Andrew's Day [Scotland] |St. Andrew's Day            |false        |GB               |2020-11-30 00:00:00|
|Portugal       |Restauração da Independência|Restauração da Independência|null         |PT             

## Write data to the default ADLS Gen2 storage

We are going to write the spark dateframe to your default ADLS Gen2 storage account.


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
account_name = 'synstorest' # fill in your primary account name
container_name = 'syn' # fill in your container name
relative_path = 'University/' # fill in your relative folder path

adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path)
print('Primary storage account path: ' + adls_path)

StatementMeta(SampleSpark, 18, 9, Finished, Available)

Primary storage account path: abfss://syn@synstorest.dfs.core.windows.net/University/

### Save a dataframe as Parquet, JSON or CSV
If you have a dataframe, you can save it to Parquet or JSON with the .write.parquet(), .write.json() and .write.csv() methods respectively.

Dataframes can be saved in any format, regardless of the input format.


In [10]:
parquet_path = adls_path + 'holiday.parquet'
json_path = adls_path + 'holiday.json'
csv_path = adls_path + 'holiday.csv'
print('parquet file path: ' + parquet_path)
print('json file path： ' + json_path)
print('csv file path: ' + csv_path)

StatementMeta(SampleSpark, 18, 10, Finished, Available)

parquet file path: abfss://syn@synstorest.dfs.core.windows.net/University/holiday.parquet
json file path： abfss://syn@synstorest.dfs.core.windows.net/University/holiday.json
csv file path: abfss://syn@synstorest.dfs.core.windows.net/University/holiday.csv

In [11]:
hol_df.write.parquet(parquet_path, mode = 'overwrite')
hol_df.write.json(json_path, mode = 'overwrite')
hol_df.write.csv(csv_path, mode = 'overwrite', header = 'true')

StatementMeta(SampleSpark, 18, 11, Finished, Available)

### Save a dataframe as text files
If you have a dataframe that you want ot save as text file, you must first covert it to an RDD and then save that RDD as a text file.


In [12]:
# Define the text file path
text_path = adls_path + 'holiday.txt'
print('text file path: ' + text_path)

StatementMeta(SampleSpark, 18, 12, Finished, Available)

text file path: abfss://syn@synstorest.dfs.core.windows.net/University/holiday.txt

In [13]:
# Covert spark dataframe into RDD 
hol_RDD = hol_df.rdd
type(hol_RDD)

StatementMeta(SampleSpark, 18, 13, Finished, Available)

<class 'pyspark.rdd.RDD'>

If you have an RDD, you can convert it to a text file like the following:


In [14]:
 # Save RDD as text file
hol_RDD.saveAsTextFile(text_path)

StatementMeta(SampleSpark, 18, 14, Finished, Available)

# Read data from the default ADLS Gen2 storage


### Create a dataframe from parquet files


In [15]:
df_parquet = spark.read.parquet(parquet_path)

StatementMeta(SampleSpark, 18, 15, Finished, Available)

### Create a dataframe from JSON files


In [17]:
df_json = spark.read.json(json_path)
df_json.show()

StatementMeta(SampleSpark, 18, 17, Finished, Available)

+---------------+-----------------+--------------------+--------------------+-------------+--------------------+
|countryOrRegion|countryRegionCode|                date|         holidayName|isPaidTimeOff|normalizeHolidayName|
+---------------+-----------------+--------------------+--------------------+-------------+--------------------+
|         Norway|               NO|2020-11-29T00:00:...|              Søndag|         null|              Søndag|
|         Sweden|               SE|2020-11-29T00:00:...|              Söndag|         null|              Söndag|
|       Scotland|             null|2020-11-30T00:00:...|    St. Andrew's Day|         null|    St. Andrew's Day|
| United Kingdom|               GB|2020-11-30T00:00:...|St. Andrew's Day ...|        false|    St. Andrew's Day|
|       Portugal|               PT|2020-12-01T00:00:...|Restauração da In...|         null|Restauração da In...|
|        Finland|               FI|2020-12-06T00:00:...|    Itsenäisyyspäivä|         null|    I

### Create a dataframe from CSV files


In [18]:
df_csv = spark.read.csv(csv_path, header = 'true')
df_csv.show()

StatementMeta(SampleSpark, 18, 18, Finished, Available)

+---------------+--------------------+--------------------+-------------+-----------------+--------------------+
|countryOrRegion|         holidayName|normalizeHolidayName|isPaidTimeOff|countryRegionCode|                date|
+---------------+--------------------+--------------------+-------------+-----------------+--------------------+
|         Norway|              Søndag|              Søndag|         null|               NO|2020-11-29T00:00:...|
|         Sweden|              Söndag|              Söndag|         null|               SE|2020-11-29T00:00:...|
|       Scotland|    St. Andrew's Day|    St. Andrew's Day|         null|             null|2020-11-30T00:00:...|
| United Kingdom|St. Andrew's Day ...|    St. Andrew's Day|        false|               GB|2020-11-30T00:00:...|
|       Portugal|Restauração da In...|Restauração da In...|         null|               PT|2020-12-01T00:00:...|
|        Finland|    Itsenäisyyspäivä|    Itsenäisyyspäivä|         null|               FI|2020-

### Create an RDD from text file


In [19]:
text = sc.textFile(text_path)
print(text)

StatementMeta(SampleSpark, 18, 19, Finished, Available)

abfss://syn@synstorest.dfs.core.windows.net/University/holiday.txt MapPartitionsRDD[59] at textFile at NativeMethodAccessorImpl.java:0